In [1]:
from importlib import reload
def reload_all():
    import tsm; reload(tsm)
    import _keys; reload(_keys)
    import blizzard; reload(blizzard)
    import config; reload(config)
    import kvstore; reload(kvstore)
    import snapshot; reload(snapshot)
    import elemental_arbitrage; reload(elemental_arbitrage)
    import bliz_tsm_join; reload(bliz_tsm_join)
    import crafting; reload(crafting)
    import kvstore; reload(kvstore)
    import crafting; reload(crafting)
    import procurement; reload(procurement)
reload_all()
from itables import show

In [2]:
from snapshot import SnapshotProcessor
from tsm import auction_house_snapshot
from config import tsm_region_id
from config import tsm_realm_id
from config import tsm_ah_id
from config import tsm_cache_dir

def tsm_ah_snapper():
    return auction_house_snapshot(tsm_region_id, tsm_realm_id, tsm_ah_id)

tsm_ah_snap = SnapshotProcessor(tsm_ah_snapper, cache_dir=tsm_cache_dir)
tsm_ah = tsm_ah_snap.get(max_age_seconds=3000)

In [3]:
from snapshot import SnapshotProcessor
from blizzard import auction_data
from blizzard import auction_summary
from config import blizzard_realm_id
from config import blizzard_ah_id
from config import blizzard_cache_dir

def bliz_ah_snapper():
    return auction_data(blizzard_realm_id, blizzard_ah_id)

bliz_ah_snap = SnapshotProcessor(bliz_ah_snapper, cache_dir=blizzard_cache_dir)
bliz_ah = bliz_ah_snap.get(max_age_seconds=3000)

In [4]:
from kvstore import InefficientKVStore
from blizzard import ItemLookup
from blizzard import collapse_languages
from config import blizzard_item_cache
from config import blizzard_item_reverse_cache

items = ItemLookup(
    InefficientKVStore(blizzard_item_cache),
    InefficientKVStore(blizzard_item_reverse_cache),
)

In [5]:
from bliz_tsm_join import item_info_getter
item_info = item_info_getter(items, bliz_ah_snap, tsm_ah_snap, max_age_seconds=3000)
item_info_current = item_info_getter(items, bliz_ah_snap, tsm_ah_snap, max_age_seconds=0)

In [6]:
from cytoolz import groupby
import pandas as pd

df_sales = pd.read_csv("Accounting_Earthfury_sales.csv")
df_sales["mode"] = "sold"
df_purchases = pd.read_csv("Accounting_Earthfury_purchases.csv")
df_purchases["mode"] = "purchased"
df_expired = pd.read_csv("Accounting_Earthfury_expired.csv")
df_expired["mode"] = "expired"
df_canceled = pd.read_csv("Accounting_Earthfury_canceled.csv")
df_canceled["mode"] = "canceled"
df_all = pd.concat([df_sales, df_purchases, df_expired, df_canceled])

entries = list(df_all.transpose().to_dict().values())
grouped = groupby(lambda entry: entry["itemString"], entries)
names = {entry["itemString"]: entry["itemName"] for entry in entries}
nets = {
    k: sum(
        (
            vv["price"]*vv["quantity"] if vv["mode"] == "sold" else
            -vv["price"]*vv["quantity"] if vv["mode"] == "purchased" else
            0
        ) for vv in v
    )
    for (k, v) in grouped.items()
}
ah_events = {k: [vv["mode"] for vv in v if vv["source"] != "Vendor"] for (k, v) in grouped.items()}
sale_rate = {
    k: v.count("sold") / (v.count("sold") + v.count("expired") + v.count("canceled"))
    for (k, v) in ah_events.items()
    if v.count("sold") > 1
}
agg = {k: {"name": names[k], "net": nets[k], "rate": sale_rate[k], "num": len(ah_events[k])} for k in sale_rate}
show(pd.DataFrame(sorted(
    [v for v in agg.values() if v["num"] > 4],
    key=lambda entry: entry["net"],
    reverse=True,
)))

<ipython-input-6-e5b0eba07282>:14: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  entries = list(df_all.transpose().to_dict().values())


In [7]:
import pickle
from cytoolz import get as pick_from_dict
from cytoolz import sliding_window
import glob
import itertools
from blizzard import auction_summary
import re
import numpy as np
import datetime
np.set_printoptions(suppress=True)

_names = """
flask of the frost wyrm
flask of endless rage
flask of stoneblood
titanium bar
indestructible potion
eternal shadow
oily blackmouth
elixir of mighty thoughts
frostweave cloth
wildvine
felcloth
lesser healing potion
runic mana potion
greater planar essence
netherweave cloth
greater healing potion
crystallized water
purple lotus
bruiseweed
arcane dust
tiger lily
crystallized fire
major healing potion
potion of speed
potion of wild magic
lichbloom
pygmy oil
adder's tongue
imbued vial
icethorn
frost lotus
enchanted vial
goldclover
crystallized life
eternal life
deadnettle
elixir of accuracy
lesser planar essence
eternal fire
eternal water
chaotic skyflare diamond
skyflare diamond
potent monarch topaz
monarch topaz
runed scarlet ruby
scarlet ruby
purified twilight opal
twilight opal
potion of speed
potion of wild magic
lichbloom
frostweave cloth
infinite dust
greater cosmic essence
dream shard
eternal belt buckle
titansteel bar
titansteel spellblade
mageweave cloth
rough stone
copper bar
coarse stone
linen cloth
bronze bar
heavy stone
moss agate
wool cloth
medium leather
steel bar
solid stone
mithril bar
mageweave cloth
dense stone
thorium bar
runecloth
fel iron bar
mote of earth
mote of fire
netherweave cloth
adamantite bar
cobalt bar
frostweave cloth
crystallized water
crystallized earth
crystallized fire
borean leather
eternal shadow
saronite bar
""".strip().splitlines()

desired_names = list(set(_names))
desired_ids = [items.get_id(name) for name in desired_names if not name.startswith("#")]


date_re = re.compile(r"202.-..-..T..-..-..")
def _timestamp_from_name(name):
    cap = date_re.search(name).group(0)
    return datetime.datetime.strptime(cap, "%Y-%m-%dT%H-%M-%S").timestamp()


by_item = {}
timeseries = []

for snapshot in sorted(glob.glob("bliz-ah/*")):
    with open(snapshot, "rb") as f:
        entries = pickle.load(f).values()
        timeseries.extend(
            [
                {"timestamp": _timestamp_from_name(snapshot), "item_name": items.get_name(item_id), **auction_summary(item_data)}
                for (item_id, item_data) in zip(desired_ids, entries)
            ]
        )
        for (item_id, item_data) in zip(desired_ids, entries):
            name = items.get_name(item_id)
            by_item[name] = by_item.get(name, []) + [{"timestamp": date_re.search(snapshot).group(0), "item_name": items.get_name(item_id), **auction_summary(item_data)}]

def timeseries_of(key, default=np.nan):
    def _timeseries_of(item_name):
        return np.array([v.get(key, default) for v in by_item[item_name]])
    return _timeseries_of

def omit_outliers(series, sigma=2, window=5):
    mavg = [np.average(w) for w in sliding_window(window, series)]
    std = np.std(mavg)
    return np.array([x if np.abs(x) < sigma*std else np.nan for x in mavg])

def avg_finite(arr):
    return np.average(arr[np.isfinite(arr)])

In [8]:
import plotly.express as px
for yvalue in ["num", "min", "p50", "d_num", "d_min", "d_p50"]:
    df = pd.DataFrame(timeseries)
    df = df.sort_values(by="timestamp")

    if yvalue.startswith("d_"):
        dfg = df.pivot(index="timestamp", columns="item_name", values=yvalue[2:])
        dfg = dfg.diff().div(dfg.index.to_series().diff(), axis=0)
    else:
        dfg = df.pivot(index="timestamp", columns="item_name", values=yvalue)

    fig = px.scatter(dfg, title=yvalue, log_y=False)
    fig.write_html(f"{yvalue}.html")

In [9]:
def min_series(x):
    return omit_outliers(timeseries_of("avg_sell")(x), sigma=0.5, window=10)

wild_magic_pot = avg_finite(min_series("potion of wild magic") - (2*min_series("lichbloom") + min_series("pygmy oil") + np.array(1.8)))
speed_pot = avg_finite(min_series("potion of speed") - (2*min_series("adder's tongue") + min_series("pygmy oil") + np.array(1.8)))
wyrm = avg_finite(2*min_series("flask of the frost wyrm") - (5*min_series("icethorn") + 5*min_series("lichbloom") + min_series("frost lotus") + np.array(5)))
rage = avg_finite(2*min_series("flask of endless rage") - (5*min_series("goldclover") + 7*min_series("lichbloom") + min_series("frost lotus") + np.array(5)))
stoneblood = avg_finite(2*min_series("flask of stoneblood") - (3*min_series("crystallized life") + 7*min_series("lichbloom") + min_series("frost lotus") + np.array(5)))
indestructible = avg_finite(2*min_series("indestructible potion") - (2*min_series("icethorn") + np.array(1.8)))

In [10]:
def item_infos(items):
    return pd.DataFrame([item_info(i) for i in items])

In [11]:
ii = item_infos(_names)
show(ii)

In [12]:
from crafting import Recipes
from crafting import purchase_price
from crafting import crafting_price
from crafting import gold


r = Recipes(items)
i = r.ingredients

def buy(item, price_method=gold("realm_market_value")):
    return purchase_price(item_info, item, price_method)


def buy_now(item):
    return buy(item, price_method=gold("min"))


def buy_market(item):
    return buy(item, price_method=gold("realm_market_value"))


recipe_listing = """
2 flask of the frost wyrm
5 icethorn + 5 lichbloom + frost lotus + enchanted vial

2 flask of endless rage
7 lichbloom + 3 goldclover + frost lotus + enchanted vial

2 flask of stoneblood
7 lichbloom + 3 crystallized life + frost lotus + enchanted vial

firecracker salmon
glacial salmon + northern spices

elixir of greater agility
sungrass + goldthorn + crystal vial

titanium bar
8 saronite bar

titanium bar
2 titanium ore

titansteel bar
3 titanium bar + eternal fire + eternal earth + eternal shadow

indestructible potion
2 icethorn + imbued vial

saronite bar
2 saronite ore

potion of speed
2 adder's tongue + pygmy oil + imbued vial

potion of wild magic
2 lichbloom + pygmy oil + imbued vial

titansteel spellblade
6 titansteel bar + 6 infinite dust + 2 frozen orb

titansteel bar
3 titanium bar + eternal fire + eternal earth + eternal shadow

glyph of wild growth
resilient parchment + ethereal ink

spellweave robe
8 spellweave + 6 bolt of imbued frostweave + eternium thread + frozen orb

spellweave
bolt of imbued frostweave + 2 eternal fire

bolt of imbued frostweave
2 bolt of frostweave + 2 infinite dust

bolt of frostweave
5 frostweave cloth

earthsiege diamond
dark jade + huge citrine + eternal fire

skyflare diamond
bloodstone + chalcedony + eternal air

noise machine
2 froststeel tube + 2 overcharged capacitor + 8 handful of cobalt bolts

froststeel tube
8 cobalt bar + crystallized water

overcharged capacitor
4 cobalt bar + crystallized earth

handful of cobalt bolts
2 cobalt bar

cobalt bar
cobalt ore

mechanized snow goggles
8 saronite bar + 2 borean leather + eternal shadow

saronite bar
2 saronite ore

wormhole generator: northrend
8 titanium bar + 2 eternal shadow + 2 eternal fire + 2 eternal water + 2 eternal air
"""

reversible_recipes = """
eternal shadow
10 crystallized shadow

eternal water
10 crystallized water

eternal fire
10 crystallized fire

eternal air
10 crystallized air

eternal life
10 crystallized life

greater planar essence
3 lesser planar essence

greater cosmic essence
3 lesser cosmic essence
"""

vendor = {
    "wild spineleaf": 0.45,
    "enchanted vial": 0.90,
    "imbued vial": 0.36,
}


r_out = None
for line in recipe_listing.splitlines():
    if line.strip() and not line.strip().startswith("#"):
        if r_out:
            r.recipe_from_strings(r_out, line.strip())
            r_out = None
        else:
            r_out = line.strip()

            
r_out = None
for line in reversible_recipes.splitlines():
    if line.strip() and not line.strip().startswith("#"):
        if r_out:
            r.recipe_from_strings(r_out, line.strip())
            r.recipe_from_strings(line.strip(), r_out)
            r_out = None
        else:
            r_out = line.strip()

inventory = {
    "frost lotus": (39+37),
    "goldclover": 144,
    "eternal life": 156,
    "crystallized life": 5,
    "eternal shadow": 16,
    "northern spices": 7,
}

from procurement import ProcurementPlanner
from procurement import topk_procurements
pp = ProcurementPlanner(items, item_info, r, vendor, approaches=["ah_buy_now", "craft", "vendor"])

In [13]:
buy_now(i("earthsiege diamond"))

48.0

In [14]:
topk_procurements(pp, i("earthsiege diamond"), k=3)

[(23.614,
  [Craft(item=earthsiege diamond, ingredients=huge citrine + dark jade + eternal fire),
   AHBuyNow(item=huge citrine, gold=1.6327),
   AHBuyNow(item=dark jade, gold=0.8721),
   AHBuyNow(item=eternal fire, gold=21.1092)]),
 (23.614,
  [Craft(item=earthsiege diamond, ingredients=huge citrine + dark jade + eternal fire),
   AHBuyNow(item=huge citrine, gold=1.6327),
   AHBuyNow(item=dark jade, gold=0.8721),
   Craft(item=eternal fire, ingredients=10.0*crystallized fire),
   Craft(item=10.0*crystallized fire, ingredients=eternal fire),
   AHBuyNow(item=eternal fire, gold=21.1092)]),
 (24.961799999999997,
  [Craft(item=earthsiege diamond, ingredients=huge citrine + dark jade + eternal fire),
   AHBuyNow(item=huge citrine, gold=1.6327),
   AHBuyNow(item=dark jade, gold=0.8721),
   Craft(item=eternal fire, ingredients=10.0*crystallized fire),
   AHBuyNow(item=10.0*crystallized fire, gold=22.456999999999997)])]

In [15]:
buy_now(i("skyflare diamond"))

35.0

In [16]:
topk_procurements(pp, i("skyflare diamond"))

[(9.1791,
  [Craft(item=skyflare diamond, ingredients=chalcedony + bloodstone + eternal air),
   AHBuyNow(item=chalcedony, gold=1.5602),
   AHBuyNow(item=bloodstone, gold=1.6582),
   AHBuyNow(item=eternal air, gold=5.9607)]),
 (9.1791,
  [Craft(item=skyflare diamond, ingredients=chalcedony + bloodstone + eternal air),
   AHBuyNow(item=chalcedony, gold=1.5602),
   AHBuyNow(item=bloodstone, gold=1.6582),
   Craft(item=eternal air, ingredients=10.0*crystallized air),
   Craft(item=10.0*crystallized air, ingredients=eternal air),
   AHBuyNow(item=eternal air, gold=5.9607)]),
 (12.1114,
  [Craft(item=skyflare diamond, ingredients=chalcedony + bloodstone + eternal air),
   AHBuyNow(item=chalcedony, gold=1.5602),
   AHBuyNow(item=bloodstone, gold=1.6582),
   Craft(item=eternal air, ingredients=10.0*crystallized air),
   AHBuyNow(item=10.0*crystallized air, gold=8.893)]),
 (35.0, [AHBuyNow(item=skyflare diamond, gold=35.0)])]

In [17]:
topk_procurements(pp, i("noise machine"))

[(25.6996, [AHBuyNow(item=noise machine, gold=25.6996)]),
 (49.943380000000005,
  [Craft(item=noise machine, ingredients=2.0*froststeel tube + 2.0*overcharged capacitor + 8.0*handful of cobalt bolts),
   Craft(item=2.0*froststeel tube, ingredients=2.0*crystallized water + 16.0*cobalt bar),
   Craft(item=2.0*crystallized water, ingredients=0.2*eternal water),
   AHBuyNow(item=0.2*eternal water, gold=0.6997800000000001),
   Craft(item=24.0*cobalt bar, ingredients=24.0*cobalt ore),
   AHBuyNow(item=24.0*cobalt ore, gold=35.251200000000004),
   Craft(item=2.0*overcharged capacitor, ingredients=2.0*crystallized earth + 8.0*cobalt bar),
   AHBuyNow(item=2.0*crystallized earth, gold=1.9972),
   AHBuyNow(item=8.0*handful of cobalt bolts, gold=11.9952)]),
 (50.47580000000001,
  [Craft(item=noise machine, ingredients=2.0*froststeel tube + 2.0*overcharged capacitor + 8.0*handful of cobalt bolts),
   Craft(item=2.0*froststeel tube, ingredients=2.0*crystallized water + 16.0*cobalt bar),
   AHBuyNow

In [18]:
topk_procurements(pp, i("mechanized snow goggles"))

[(13.9444,
  [Craft(item=mechanized snow goggles, ingredients=eternal shadow + 2.0*borean leather + 8.0*saronite bar),
   AHBuyNow(item=eternal shadow, gold=2.248),
   AHBuyNow(item=2.0*borean leather, gold=0.5796),
   Craft(item=8.0*saronite bar, ingredients=16.0*saronite ore),
   AHBuyNow(item=16.0*saronite ore, gold=11.1168)]),
 (13.9444,
  [Craft(item=mechanized snow goggles, ingredients=eternal shadow + 2.0*borean leather + 8.0*saronite bar),
   AHBuyNow(item=eternal shadow, gold=2.248),
   AHBuyNow(item=2.0*borean leather, gold=0.5796),
   Craft(item=8.0*saronite bar, ingredients=16.0*saronite ore),
   AHBuyNow(item=16.0*saronite ore, gold=11.1168)]),
 (13.9444,
  [Craft(item=mechanized snow goggles, ingredients=eternal shadow + 2.0*borean leather + 8.0*saronite bar),
   Craft(item=eternal shadow, ingredients=10.0*crystallized shadow),
   Craft(item=10.0*crystallized shadow, ingredients=eternal shadow),
   AHBuyNow(item=eternal shadow, gold=2.248),
   AHBuyNow(item=2.0*borean lea

In [19]:
earthsiege = topk_procurements(pp, i("earthsiege diamond"), k=10)
skyflare = topk_procurements(pp, i("skyflare diamond"), k=10)

In [20]:
topk_procurements(pp, i("15 crystallized life"))

[(17.999850000000002,
  [Craft(item=15*crystallized life, ingredients=1.5*eternal life),
   AHBuyNow(item=1.5*eternal life, gold=17.999850000000002)]),
 (20.955000000000002,
  [AHBuyNow(item=15*crystallized life, gold=20.955000000000002)]),
 (20.955000000000002,
  [Craft(item=15*crystallized life, ingredients=1.5*eternal life),
   Craft(item=1.5*eternal life, ingredients=15.0*crystallized life),
   AHBuyNow(item=15.0*crystallized life, gold=20.955000000000002)])]

In [21]:
skyflare

[(9.1791,
  [Craft(item=skyflare diamond, ingredients=chalcedony + bloodstone + eternal air),
   AHBuyNow(item=chalcedony, gold=1.5602),
   AHBuyNow(item=bloodstone, gold=1.6582),
   AHBuyNow(item=eternal air, gold=5.9607)]),
 (9.1791,
  [Craft(item=skyflare diamond, ingredients=chalcedony + bloodstone + eternal air),
   AHBuyNow(item=chalcedony, gold=1.5602),
   AHBuyNow(item=bloodstone, gold=1.6582),
   Craft(item=eternal air, ingredients=10.0*crystallized air),
   Craft(item=10.0*crystallized air, ingredients=eternal air),
   AHBuyNow(item=eternal air, gold=5.9607)]),
 (12.1114,
  [Craft(item=skyflare diamond, ingredients=chalcedony + bloodstone + eternal air),
   AHBuyNow(item=chalcedony, gold=1.5602),
   AHBuyNow(item=bloodstone, gold=1.6582),
   Craft(item=eternal air, ingredients=10.0*crystallized air),
   AHBuyNow(item=10.0*crystallized air, gold=8.893)]),
 (35.0, [AHBuyNow(item=skyflare diamond, gold=35.0)])]

In [22]:
topk_procurements(pp, i("10 potion of speed"))

[(21.951999999999998,
  [Craft(item=10*potion of speed, ingredients=10.0*imbued vial + 20.0*adder's tongue + 10.0*pygmy oil),
   VendorBuy(item=10.0*imbued vial, who='vendor', where='somewhere', gold=0.36),
   AHBuyNow(item=20.0*adder's tongue, gold=18.592),
   AHBuyNow(item=10.0*pygmy oil, gold=3.0)]),
 (47.617999999999995,
  [Craft(item=10*potion of speed, ingredients=10.0*imbued vial + 20.0*adder's tongue + 10.0*pygmy oil),
   AHBuyNow(item=10.0*imbued vial, gold=26.025999999999996),
   AHBuyNow(item=20.0*adder's tongue, gold=18.592),
   AHBuyNow(item=10.0*pygmy oil, gold=3.0)]),
 (117.5, [AHBuyNow(item=10*potion of speed, gold=117.5)])]

In [23]:
topk_procurements(pp, i("10 potion of wild magic"))

[(42.26,
  [Craft(item=10*potion of wild magic, ingredients=10.0*imbued vial + 10.0*pygmy oil + 20.0*lichbloom),
   VendorBuy(item=10.0*imbued vial, who='vendor', where='somewhere', gold=0.36),
   AHBuyNow(item=10.0*pygmy oil, gold=3.0),
   AHBuyNow(item=20.0*lichbloom, gold=38.9)]),
 (67.92599999999999,
  [Craft(item=10*potion of wild magic, ingredients=10.0*imbued vial + 10.0*pygmy oil + 20.0*lichbloom),
   AHBuyNow(item=10.0*imbued vial, gold=26.025999999999996),
   AHBuyNow(item=10.0*pygmy oil, gold=3.0),
   AHBuyNow(item=20.0*lichbloom, gold=38.9)]),
 (87.99799999999999,
  [AHBuyNow(item=10*potion of wild magic, gold=87.99799999999999)])]

In [24]:
catalog = [
    i("5 indestructible potion"),
    i("5 potion of speed"),
    i("5 flask of stoneblood"),
    i("5 flask of the frost wyrm"),
    i("5 flask of pure mojo"),
    i("5 flask of endless rage"),
    i("earthsiege diamond"),
    i("skyflare diamond"),
]

products = []
for it in catalog:
    _cprice = buy_now(it)
    _methods = topk_procurements(pp, it, k=3)
    _best_profit = _cprice - _methods[0][0]
    products.append(
        {
            "item": it,
            "best_profit": _best_profit,
            "current_price": _cprice,
            "methods": _methods,
        }
    )
sorted(products, key=lambda x: x["best_profit"], reverse=True)

[{'item': 5*potion of speed,
  'best_profit': 47.594,
  'current_price': 58.75,
  'methods': [(11.155999999999999,
    [Craft(item=5*potion of speed, ingredients=5.0*imbued vial + 10.0*adder's tongue + 5.0*pygmy oil),
     VendorBuy(item=5.0*imbued vial, who='vendor', where='somewhere', gold=0.36),
     AHBuyNow(item=10.0*adder's tongue, gold=9.296),
     AHBuyNow(item=5.0*pygmy oil, gold=1.5)]),
   (23.808999999999997,
    [Craft(item=5*potion of speed, ingredients=5.0*imbued vial + 10.0*adder's tongue + 5.0*pygmy oil),
     AHBuyNow(item=5.0*imbued vial, gold=13.012999999999998),
     AHBuyNow(item=10.0*adder's tongue, gold=9.296),
     AHBuyNow(item=5.0*pygmy oil, gold=1.5)]),
   (58.75, [AHBuyNow(item=5*potion of speed, gold=58.75)])]},
 {'item': skyflare diamond,
  'best_profit': 25.8209,
  'current_price': 35.0,
  'methods': [(9.1791,
    [Craft(item=skyflare diamond, ingredients=chalcedony + bloodstone + eternal air),
     AHBuyNow(item=chalcedony, gold=1.5602),
     AHBuyNow(it